In [1]:
# Remove emails.txt from HDFS, if it already exists
!hdfs dfs -rm emails.txt

# Upload emails.txt into HDFS from the local file system
!hdfs dfs -put emails.txt

24/11/18 09:07:14 INFO fs.TrashPolicyDefault: Moved: 'hdfs://BigDataHa/user/garza/emails.txt' to trash at: hdfs://BigDataHa/user/garza/.Trash/Current/user/garza/emails.txt


In [2]:
# For debug purposes
# Show the content of the input file
!hdfs dfs -cat emails.txt

paolo.garza@polito.it
test_polito.it
wrongformat.polito.it
luca.cagliero@polito.it
daniele.apiletti@polito.it


In [3]:
inputPath = "emails.txt"

In [4]:
# Define an accumulator. Initialize it to 0
invalidEmails = sc.accumulator(0)

In [5]:
# Read the content of the input textual file
#
# cache is applied on the output of textFile

emailsRDD = sc.textFile(inputPath)\
                .cache()

In [6]:
# Define the filtering function
# Select the emails that contain @
# Update the accumulator if @ is not present in the input string/email
def validEmailFunc(line):
    if (line.find('@')<0):
        invalidEmails.add(1)
        return False
    else:
        return True

In [7]:
# Select only valid emails
# Count also the number of invalid emails using an accumulator
validEmailsRDD = emailsRDD.filter(validEmailFunc)

In [8]:
# Print the list of valid emails on the standard output
# I suppose validEmailsRDD is small and collect does not return a large list
print(validEmailsRDD.collect())

[Stage 0:>                                                          (0 + 2) / 2]

['paolo.garza@polito.it', 'luca.cagliero@polito.it', 'daniele.apiletti@polito.it']


In [9]:
# Print the number of valid emails on the standard output
print("Valid email addresses: ", validEmailsRDD.count())

Valid email addresses:  3


In [10]:
# Print the number of invalid emails on the standard output
print("Invalid email addresses: ", invalidEmails.value)

Invalid email addresses:  4


In [11]:
# This code is similar to that reported in ExampleAccumulators_Example3.ipynb
# The only difference is that the cache is used after the textFile method.
# The conte of the input file is small enough to be stored in the servers' main memory. 


# The first action (collect) triggers the first read of the file, the execution of the textFile and filter 
# transformations, and the content of emailsRDD is cached. 

# The second action (count) is computed by reading the content of emailsRDD from the servers' main memory.
# The original input file is no longer needed (its content is already in the cached RDD emailsRDD), but the filter 
# part is executed again because we cached emailsRDD instead of validEmailsRDD in this code.

# The accumulator's value is wrong (because the filter transformation is executed twice on 
# the input values read from emailsRDD).
# The input file is read only once.